# **Swyp Teddy Bear Club NFT Rewards Calculation**

## **1. Overview**

An **Initial NFT Offering (INO)** is a fundraising method in the cryptocurrency and blockchain ecosystem, specifically tailored for NFTs (Non-Fungible Tokens). Similar to an Initial Coin Offering (ICO), INOs allow creators, projects, or platforms to raise funds by selling a collection of NFTs to early adopters and investors.

In this context, the **Teddy Bear Club NFTs** served as **TeddySwap's NFT offering**. By minting these NFTs, owners earn rewards in **TEDY Tokens**, with the amount determined by both the rarity and quantity of the NFTs they hold. The INO was conducted in two rounds:
- [**Round 1**](https://medium.com/@TeddySwapDEX/teddy-bears-club-minting-utility-and-launch-date-23e9a4e4446d) launched on **February 17, 2023**
- [**Round 2**](https://medium.com/@TeddySwapDEX/teddy-bears-club-round-2-c209556b379f) launched on **March 8, 2023**

### *Note!*
In TeddySwap's [**initial tokenomics**](https://medium.com/@TeddySwapDEX/teddyswap-tokenomics-fcd700aea552), the market cap was set at **5,000,000,000 TEDY**. However, it was later adjusted to **8,000,000 TEDY** in a [**second announcement**](https://medium.com/@TeddySwapDEX/tedy-tokenomics-v2-fc92723ead0c). This notebook utilizes the original market cap for the calculations, scaling it to the new market cap in the final step.

Additionally, the Swyp Team considers rewards for holders of the tokens until Janurary 17, 2024


### **1.1 Purpose**

This document outlines the process for calculating the NFTs' corresponding TEDY token rewards for the Teddy Bear Club NFTs across Round 1 and Round 2. Since the ranking and reward structures differ between the two rounds, separate functions calculate rewards for each round. They are then summed to determine the total rewards, where applicable. 

This document aims to provide a comprehensive explanation of how the token rewards are computed, ensuring transparency and reproducibility for NFT holders.

Feel free to download this notebook and explore it at your convenience.

---

### **1.2 NFT Information**

### *Note!*

Throughout the TEDY life cycle and the duration of the INO, several announcements were released that affected the total TEDY rewards initially determined. Unfortunately, the official Teddy Bears Club can no longer be accessed. The link to those are from the web archive. Below is a summary of these announcements:

1. According to the official [Teddy Bears Club](https://webcf.waybackmachine.org/web/20240905102756/https://tbc.teddyswap.org/) website, the TEDY earnings were updated to include an additional **10% bonus** to the initial values decided in the [initial announcement](https://medium.com/@TeddySwapDEX/teddy-bears-club-minting-utility-and-launch-date-23e9a4e4446d). 

Additionally, as **804** Round 1 NFTs were sold, the tiers were adjusted based on the [source code](https://github.com/teddy-swap/tbc-homepage/blob/main/src/App.tsx) of the official Teddy Bear Club website.

### Round 1

- **Policy ID:** `ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d87`
- **Total NFTs:** 804
- **Tier System:**

| **Tier** | **Initial Rarity Range** | **Initial TEDY Allocation** | **Updated Rarity Range** | **Updated TEDY Allocation** |
|----------|--------------------------|-----------------------------|----------------------|-------------------|
| Tier 1   | Rarity 1 - 100     | 28,000  | Rarity 1 - 15    | 30,800                                  |
| Tier 2   | Rarity 101 - 999   | 17,500  | Rarity 16 - 100  | 19,250                                  |
| Tier 3   | Rarity 1,000 - 2,999  | 12,600  | Rarity 101 - 250 | 13,860                                  |
| Tier 4   | Rarity 3,000 - 5,999  | 11,200  | Rarity 251 - 500 | 12,320                                  |
| Tier 5   | Rarity 6,000 - 10,000  | 10,500  | Rarity 501 - 804 | 11,550                                  |

2. The [**initial announcement**](https://medium.com/@TeddySwapDEX/teddy-bears-club-round-2-c209556b379f) regarding **Round 2** detailed tiers for ranks **1 - 9,200**. Since **2,035** NFTs were sold, the ranks were adjusted accordingly in this [**Discord announcement**](https://discord.com/channels/1053191459597201448/1053207517154508870/1095880926086570105) on the Discord server.

### Round 2

- **Policy ID:** `da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c`
- **Total NFTs:** 2,035
- **Tier System:**

| **Tier** | **Initial Rarity Range** | **Updated Rarity Range** | **Previous MarketCap TEDY** |
|----------|-------------------|-------------------------------|---------------------|
| Tier 1   | Rarity 1 - 100       |  Rarity 1 - 23             |       7,000         |
| Tier 2   | Rarity 101 - 999     |  Rarity 24 - 221           |      6,000          |
| Tier 3   | Rarity 1,000 - 2,999 |  Rarity 222 - 664          |        5,000        |
| Tier 4   | Rarity 3,000 - 5,999 |  Rarity 665 - 1,327        |        4,500        |
| Tier 5   | Rarity 6,000 - 9,200 |  Rarity 1,328 - 2,035      |         4,200       |

3. Additionally, the [Round 2 announcement](https://medium.com/@TeddySwapDEX/teddy-bears-club-round-2-c209556b379f) established that unsold NFTs from the INO would contribute to a surplus reward pool. Initially, these TEDY rewards were to be evenly distributed to NFT holders in proportion to the number of NFTs they own.

However, due to the large quantity of remaining tokens diluting the ranking system's impact, we  calculate surplus rewards per tier, with a corresponding percentage for each tier. Calculations for these values are explain below in 3.4

| **Tier** | **Percentage** | **Round 1 TEDY** | **Round 2 TEDY** |
|----------|----------------|------------------|------------------|
| Tier 1   | 10%            | 654,932          | 238,157          |
| Tier 2   | 15%            | 146,196          | 53,162           |
| Tier 3   | 20%            | 98,376           | 35,773           |
| Tier 4   | 27%            | 84,128           | 30,592           |
| Tier 5   | 28%            | 76,310           | 27,749           |

4. During the Round 2 NFTs, the [Teddy Bear Surprise](https://medium.com/@TeddySwapDEX/1st-teddy-bear-surprise-945ed9e7405a) was released. Several succeeding announcements were releases on discord, with the second table below containing the updated results.

Initial Announcement:

| **Tier** | **Range** | **TEDY Rewards** 
|----------|----------------|------------|
| Tier 1   | 1-1,250        | 10,733     | 
| Tier 2   | 1,251 - 1,400  | 9,733      | 
| Tier 3   | 1,401 - 1,600  | 8,400      |
| Tier 4   | 1,601 - 2,000  | 6,400      | 
| Tier 5   | 2,001 - 2,500  | 4,000      | 

Post Succeeding Announcements:

1. [+8,400 TEDY for 1-1,599](https://discord.com/channels/1053191459597201448/1053207517154508870/1092528784198094949)
2. [Tier Modification](https://discord.com/channels/1053191459597201448/1053207517154508870/1093526335999197274)
3. [+2,000 TEDY for 1,800 - 2,000](https://discord.com/channels/1053191459597201448/1053207517154508870/1093690327631659049)
4. [+2,000 TEDY for 1,881-2,000](https://discord.com/channels/1053191459597201448/1053207517154508870/1093950972524507278)

| **Tier** | **Range** | **TEDY Rewards** 
|----------|-----------|-----------------|
| Tier 1   | 1-1,250        | 19,133     | 
| Tier 2   | 1,251 - 1,400  | 18,133     | 
| Tier 3   | 1,401 - 1,599  | 16,800     |
| Tier 4   | 1,600          | 8,400      | 
| Tier 5   | 1,601 - 1,799  | 6,400      | 
| Tier 6   | 1,800          | 8,400      | 
| Tier 7   | 1,801 - 1,880  | 6,000      |
| Tier 8   | 1,881 - 2,000  | 8,000      | 
| Tier 9   | 2,001 - 2,035  | 4,000      | 
---

### **1.3 Workflow Summary**
The workflow is divided into the following steps:

1. **Fetch Data**  
   1.1 Load [`collection_1.json`](./data/collection_1.json) and [`collection_2.json`](./data/collection_2.json) data which was scraped and saved in [`webscraper.ipynb`](./webscraper.ipynb), as well the [`collectiontraits_1.json`](./data/collectiontraits_1.json) and [`collectiontraits_2.json`](./data/collectiontraits_2.json) computed and saved in [`attributecounter.ipynb`](attributecounter.ipynb).

2. **NFT Rank Calculation**  
   2.1 Assign a score to each trait value based on its rarity. <br>
   2.2 Compute the rarity rank for each NFT.

3. **NFT Rewards Calculation**  
   3.1 Load rank data from the [`collectionranks.csv`](./data/collectionranks.csv) created from [`rankdata.ipynb`](rankdata.ipynb)<br>
   3.2 Given a subject, identify its TEDY reward based on its round and rarity rank. <br>
   3.3 Calculate the Teddy Bear Surprise bonus TEDY for a Round 2 NFT.<br>
   3.4 Calculate an NFT's surplus TEDY reward allocation according to its tier.<br>
   3.5 Given an address, calculate the total TEDY rewards owed - utilizing the (3.2)identified tier and (3.3)surplus allocation calculated in previous steps. <br>

4. **Scale Rewards to New Market Cap**<br>
   4.1 Adjust the final TEDY token rewards from the initial [**5,000,000,000**](https://medium.com/@TeddySwapDEX/teddyswap-tokenomics-fcd700aea552) to the updated market cap of [**8,000,000**](https://medium.com/@TeddySwapDEX/tedy-tokenomics-v2-fc92723ead0c).

5. **Calculate Your Rewards**<br>
   5.1 Enter a Cardano address or a list of subjects to calculate your total TEDY rewards.
   
---

### **1.4 Necessary Package Installations and Open Statements**

The following external libraries are required:
- `Plotly.NET`
- `System.Text.Json`
- `FSharp.Data`


In [74]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json"
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"

#r "nuget: Plotly.NET.Interactive, 5.0.0"
#r "nuget: Plotly.NET.ImageExport"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet5/nuget/v3/index.json Installed Packages Plotly.NET.ImageExport, 6.1.0 Plotly.NET.Interactive, 5.0.0

In [75]:
open System.Text.Json
open System.IO
open System.Net.Http
open System.Threading.Tasks
open FSharp.Data
open FSharp.Reflection
open Plotly.NET
open Plotly.NET.Interactive
open Plotly.NET.ImageExport

# **2. Workflow**

## 1. Fetch Data
Metadata for the NFTs was scraped from the official Teddy Bears Club website ([Teddy Bears Club](https://webcf.waybackmachine.org/web/20240905102756/https://tbc.teddyswap.org/)) and [CNFT Tools](https://cnft.tools/teddybearclub2), and stored in [`collection_1.json`](./data/collection_1.json) and [`collection_2.json`](./data/collection_2.json) for Round 1 and Round 2 respectively. Please refer to the [`webscraper.ipynb`](./webscraper.ipynb) to view the process - which you may test out for yourself. 

Additionally, refer to [`attributecounter.ipynb`](./attributecounter.ipynb) for the tallying of trait value occurences, saved in [`collectiontraits_1.json`](./data/collectiontraits_1.json) and [`collectiontraits_2.json`](./data/collectiontraits_2.json).

However, the necessary metadata and trait occurrence data for the NFTs are already included in this repository as JSON files. Therefore, you may directly utilize the above mentioned files without any additional setup.

In this section, we will parse the JSON data to extract information about the NFTs and the occurrence of their traits. This data will serve as the foundation for calculating rankings and determining rewards in the subsequent sections. Using the data from the JSON files, we compute for the NFTs' rarity score and rarity rank. 

---

#### **Types**:
The NFT and NFTTraits types are partial representations of the data, as only the fields relevant to ranking and reward calculations have been included.

- **`NFT`**: Represents an individual NFT, focusing on its identifiers and traits.

- **`NftTraits`**: Captures occurrence counts of trait values across NFTs.

#### **Function**:
1. **`loadNFTs<'T>`**: 
- Loads and deserializes JSON data into a specified type:
- **Parameters**: 
     - `filePath (string)`: the path to the JSON file.
- **Returns**:
    - Parsed data of type `'T`

--- 

#### **Variables**:<br>
**`nftRound1NFTS`**: <br>
- (NFT list): NFTs from Round 1 <br>

 **`nftRound2NFTS`**:  <br>
- (NFT list):  NFTs from Round 2 <br>

 **`nftRound1Traits`**: <br>
- (NFTTraits): Trait value counts for Round 1 NFTs <br>
    
 **`nftRound2Traits`**:  <br>
- (NFTTraits): Trait value counts for Round 2 NFTs<br>

---

### 1.1 Fetch NFT Data From JSON

In [76]:
type NFT = {
    assetName: string
    encodedName: string
    Background: string
    Bear: string
    Clothes: string
    Face: string
    Handheld: string
    Head: string
    Skins: string
}

type NftTraits = {
    Background: Map<string, int>
    Bear: Map<string, int>
    Clothes: Map<string, int>
    Face: Map<string, int>
    Handheld: Map<string, int>
    Head: Map<string, int>
    Skins: Map<string, int>
}

In [77]:
let loadNfts<'T> (filePath: string) : 'T =
    let jsonContent = File.ReadAllText(filePath)
    JsonSerializer.Deserialize<'T>(jsonContent)

In [78]:
let nftRound1NFTS = loadNfts<NFT list> "./data/collection_1.json"
let nftRound2NFTS = loadNfts<NFT list> "./data/collection_2.json"

In [79]:
let nftRound1Traits = loadNfts<NftTraits> "./data/collectiontraits_1.json"
let nftRound2Traits = loadNfts<NftTraits> "./data/collectiontraits_2.json"

## **2. NFT Rank Calculation** 
Each NFT has traits such as Background, Bear, Clothing, and more, with each trait having assigned specific values. For example, the Background trait can have values like Space, Lighthouse, or Sunflowers. The overall rarity score and rank of an NFT are determined by the rarity of its trait values.

The rarity score of an NFT is calculated by evaluating the rarity of its trait values. For each trait value, we calculate a **trait score** by determining the percentage of NFTs sharing that value (trait frequency) and subtracting this percentage from 100. This ensures that rarer traits receive higher scores.

Finally, the **total rarity score** is obtained by summing the scores across all trait categories (e.g., Background, Hair, Clothes). The score is rounded to two decimal places for consistency.

$\text{Trait Score} = \text{round}\left(100.0 - \left(\frac{\text{Trait Frequency}}{\text{Total NFTs}}\right) \cdot 100.0, 2\right)$

$\text{Total Rarity Score} = \sum_{i=1}^{7} TraitScore_i$

---
### 2.1 Assign Point Values Based on Trait Value Rarity   
Using the **Trait Score** formula, we compute a score for each trait value. These scores are then stored in a map, organized by trait categories and their respective values. 

---

#### **Function**:
1. **`processNftTraits`**: 
- Calculates rarity scores for each trait value based on the trait data and total NFTs in a round.
- **Parameters**:<br>
    - **`nftTraits`**: (NftTraits): Contains the trait value occurrence data for a round.
        
    - **`totalNfts`**: (float): The total number of NFTs in the round.
- **Returns**: <br>
    - A Map<string, Map<string, float>>, where the outer map corresponds to trait categories, and the inner map holds the scores for each trait value.

---

#### **Variables**:
**`r1TotalNfts`**: 
- (float): Total NFTs in Round 1 (804.0).<br>

**`r2TotalNfts`**: 
- (float): Total NFTs in Round 2 (2035.0).<br>

**`r1ScoredTraits`**: 
- (Map<string, Map<string, float>>): A map of scored trait values for Round 1 NFTs.<br>

**`r2ScoredTraits`**: 
- (Map<string, Map<string, float>>): A map of scored trait values for Round 2 NFTs.<br>


In [80]:
let r1TotalNfts = 804.0
let r2TotalNfts = 2035.0

In [81]:
let processNftTraits (nftTraits: NftTraits) (totalNfts: float) =
    let recordType = typeof<NftTraits>
    FSharp.Reflection.FSharpType.GetRecordFields(recordType)
    |> Array.fold (fun acc field ->
        let fieldValue = field.GetValue(nftTraits) :?> Map<string, int>
        let traitScores = 
            fieldValue |> Map.map (fun key value ->
                Math.Round(100.0 - (float value / float totalNfts) * 100.0, 2)
            )
        Map.add field.Name traitScores acc
    ) Map.empty

In [82]:
let r1ScoredTraits = processNftTraits nftRound1Traits r1TotalNfts
let r2ScoredTraits = processNftTraits nftRound2Traits r2TotalNfts

### **Illustrating the Distribution of Trait Scores**: 
This section generates visual representations of the distribution of trait scores. The `createCharts` function produces doughnut charts that display each trait value along with its corresponding score.

In [83]:
let createCharts (traitScores: Map<string, Map<string, float>>) = 
    traitScores
    |> Map.fold( fun acc title labelsAndValues ->
        let labels,values = 
            labelsAndValues
            |> Map.toList
            |> List.unzip

        let combinedText = 
            List.map2 (fun label value -> sprintf "%s: %.2f" label value) labels values

        let chart = 
            Chart.Doughnut(values = values, Labels = labels, Hole = 0.5, MultiText = combinedText)
            |> Chart.withTitle title
            |> Chart.withLayout(
                Layout.init(
                    Width = 1200,
                    Height = 1000
                )
            )
                    
        chart :: acc
    )[]
    |> List.rev

let chartRound1 = createCharts r1ScoredTraits 
let chartRound2 = createCharts r2ScoredTraits

### Teddy Bear Club Round 1 Trait Value Scores

<img src="./assets/charts1/doughnut1.png" alt="Round 1 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts1/doughnut2.png" alt="Round 1 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts1/doughnut3.png" alt="Round 1 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts1/doughnut4.png" alt="Round 1 Doughnut Charts" width="1400" height="800">

In [ ]:
for chart in chartRound1 do
    display(chart |> Chart.withSize(1400,1400))

### Teddy Bears Club Round 2 Trait Value Scores

<img src="./assets/charts2/doughnut1.png" alt="Round 2 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts2/doughnut2.png" alt="Round 2 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts2/doughnut3.png" alt="Round 2 Doughnut Charts" width="1400" height="800">
<img src="./assets/charts2/doughnut4.png" alt="Round 2 Doughnut Charts" width="1400" height="800">

In [ ]:
for chart in chartRound2 do
    display(chart |> Chart.withSize(1400,1400)) 

### 2.2 Compute Overall Rarity Ranks for Each NFT. 
This section calculates the total rarity score and rank for each NFT within their respective rounds. The **total rarity score** is determined by summing the scores of all trait values associated with an NFT.

$\text{Total Rarity Score} = \sum_{i=1}^{7} TraitScore_i$

Using the results from this step, a CSV file - [`collectionranks.csv`](./data/collectionranks.csv) - was created to map NFTs to their respective ranks, enabling easier access and computation. This file is referenced in subsequent sections for calculating TEDY token rewards. For more details on generating the CSV, refer to [`rankdata.ipynb`](./rankdata.ipynb).

---

#### **Function**:
1. **`computeNFTRank`**: 
- Computes the rarity rank of NFTs based on their traits and scores.
- **Parameters**:
    - **`nftList`**: (NFT list): The list of NFTs to rank.
    - **`nftTraitScores`**: (Map<string, Map<string, float>>): Trait value scores computed above.

---

#### **Variables**:
**`rankedNFTsRound1`**: 
- (list): Stores the ranked Round 1 NFTs, including their `assetName`, `encodedName`, and `rarityScore`.

**`rankedNFTsRound2`**: 
- (list): Stores the ranked Round 2 NFTs with the same structure as above.


In [84]:
let computeNFTRank (nftList: list<NFT>) nftTraitScores = 
    nftList
    |> List.map (fun nft ->
        let rarityScore =  
            [
                ("Background", nft.Background)
                ("Bear", nft.Bear)
                ("Clothes", nft.Clothes)
                ("Face", nft.Face)
                ("Handheld", nft.Handheld)
                ("Head", nft.Head)
                ("Skins", nft.Skins)
            ]
            |> List.fold (fun acc (attrName, attrValue) ->  
                match Map.tryFind attrName nftTraitScores with
                | Some traitMap ->  
                    acc + (Map.tryFind attrValue traitMap |> Option.defaultValue 0.0)
                | None -> acc 
            ) 0.0 
        (nft.assetName, nft.encodedName, rarityScore) 
    )
    |> List.sortBy (fun (_, _, rarityScore) -> -rarityScore)
    |> List.mapi (fun index (assetName, encodedName, rarityScore) -> 
    (index + 1, assetName, encodedName, rarityScore))

In [85]:
let rankedNFTsRound1 = computeNFTRank nftRound1NFTS r1ScoredTraits
let rankedNFTsRound2 = computeNFTRank nftRound2NFTS r2ScoredTraits

## **3. NFT Rewards Calculation** 
Calculating an NFT holder's TEDY rewards involves multiple factors, including the NFTs they own, the round each NFT belongs to, and their respective tiers within those rounds. For Round 2 Teddy Bear Club NFTs, the order of minting is considered as well.

Furthermore, the calculation considers the holder's proportional share of unsold NFTs, based on the number of NFTs they hold, the NFTs's round and tier.

---
### **3.1 Load NFT rank and order data**
1. Load NFT rank data from the [`collectionranks.csv`](./data/collectionranks.csv) file in the `data` folder. The CSV was created by using the rank computations executed below. For detailed steps or to recreate the file, refer to [`rankData.ipynb`](./rankdata.ipynb).

#### **Type**:
- **`NftRecord`**: Represents an individual NFT record from the CSV.

#### **Functions**:

1. **`parseRankCsv`**:
- Parses a CSV file into a list of `NftRecord` entries.
- **Parameters**:
   - `filePath (string)`: The path to the CSV file.
- **Returns**:  
   - A `list<NftRecord>` containing records with `Name`, `Subject`, `Rank`, and `Score`.

2. **`createSubjectToRankMap`**:
- Creates a mapping of `Subject`(Policy ID + Asset Name) to `Rank`.
- **Parameters**:
   - `records (list<NftRecord>)`: A list of `NftRecord` entries.
- **Returns**:  
   - A `Map<string, int>` mapping `Subject` to `Rank`.

3. **`createNameToRankMap`**:
- Creates a mapping of `Name` (i.e. TeddyBearsClub178) to `Rank`.
- **Parameters**:
   - `records (list<NftRecord>)`: A list of `NftRecord` entries.
- **Returns**:  
   - A `Map<string, int>` mapping `Name` to `Rank`.

---

#### **Variables**:

1. **`csvPath`**:  
   - `(string)`: The file path to the CSV file containing NFT rank data.

2. **`nftDataList`**:  
   - `(list<NftRecord>)`: A list of parsed `NftRecord` entries from the CSV file.

3. **`subjectRankData`**:  
   - `(Map<string, int>)`: A mapping of `Subject` to `Rank` created using `createSubjectToRankMap`.

4. **`nameRankData`**:  
   - `(Map<string, int>)`: A mapping of `Name` to `Rank` created using `createNameToRankMap`.


In [86]:
type NftRecord = {
    Name: string
    Subject: string
    Rank: int
    Score: float
}

In [87]:
 let parseRankCsv (filePath: string) =
    File.ReadAllLines(filePath)
    |> Array.skip 1
    |> Array.map (fun line ->
        let parts = line.Split(',')
        {
            Name = parts.[0]
            Subject = parts.[1]
            Rank = int parts.[2]
            Score = float parts.[3]
        }
    )
    |> List.ofArray


In [88]:
let createSubjectToRankMap (records: NftRecord list) =
    records
    |> List.map (fun record -> record.Subject, record.Rank)
    |> Map.ofList

let createNameToRankMap (records: NftRecord list) =
    records
    |> List.map (fun record -> record.Name, record.Rank)
    |> Map.ofList

In [89]:
let rankCsvPath = "./data/collectionranks.csv"
let nftDataList = parseRankCsv rankCsvPath
let subjectRankData = createSubjectToRankMap nftDataList 
let nameRankData = createNameToRankMap nftDataList  

---

2. Load NFT order data from the ___ file in the `data` folder. The CSV was created by using the [CardanoScan API](https://docs.cardanoscan.io/). For detailed steps or to recreate the file, refer to [`api.ipynb`](./api.ipynb).

#### **Type**:
- **`OrderedNft`**: Represents an individual NFT record from the CSV.

#### **Functions**:

1. **`parseOrderCsv`**:
- Parses a CSV file into a list of `OrderedNft` entries.
- **Parameters**:
   - `filePath (string)`: The path to the CSV file.
- **Returns**:  
   - A `list<OrderedNft>` containing records with `Name`, `Subject`, `Rank`, and `Score`.

2. **`createSubjectToOrderMap`**:
- Creates a mapping of `Subject`(Policy ID + Asset Name) to `Order`.
- **Parameters**:
   - `records (list<OrderedNft>)`: A list of `OrderedNft` entries.
- **Returns**:  
   - A `Map<string, int>` mapping `Subject` to `Order`.

---

#### **Variables**:

1. **`orderCsvPath`**:  
   - `(string)`: The file path to the CSV file containing NFT order data.

2. **`nftOrderList`**:  
   - `(list<NFTRecord>)`: A list of parsed `OrderedNft` entries from the CSV file.

3. **`subjectOrderData`**:  
   - `(Map<string, int>)`: A mapping of `Subject` to `Order` created using `createSubjectToOrderMap`.

In [ ]:
type OrderedNft = {
    AssetId: string
    Order: int
}

In [ ]:
let parseOrderCsv (filePath: string) =
    File.ReadAllLines(filePath)
    |> Array.skip 1
    |> Array.map (fun line ->
        let parts = line.Split(',')
        {
            AssetId = parts.[0]
            Order = int parts.[1]
        }
    )
    |> List.ofArray

In [179]:
let createSubjectToOrderMap (records: OrderedNft list) =
    records
    |> List.map (fun record -> record.AssetId, record.Order)
    |> Map.ofList

In [ ]:
let orderCsvPath = "./data/roundtwo_tokens_minting_order.csv"
let nftOrderList = parseOrderCsv orderCsvPath
let subjectOrderData = createSubjectToOrderMap nftOrderList

### **3.2 Identify a Subject's Tier**
This section determines the tier of an NFT based on its `policyId` (indicating the round) and its rarity rank. The tier determines the base TEDY rewards allocated to the NFT.

#### **Functions**:
1. **`calculateRewardByTier`**:
- Categorizes an NFT into a tier based on its rank and round (indicated by `tiers`).
- **Parameters**:
  - `rank` (int): The rarity rank of the NFT.
  - `subject` ((int * int) list): A list of tier thresholds and their associated rewards.
- **Returns**:
  -(int): The reward corresponding to the NFT’s tier.

2. **`computeBaseReward`**:
- Determines the base TEDY rewards for a given NFT based on its round and tier.
- Calls `calculateRewardByTier` with either `round1Tiers` or `round2Tiers` based on the NFT's policy ID.
- **Parameters**:
  - `nft` (string): The unique identifier of the NFT.
  - `rankData` (Map<string, int>): A mapping of subjects to their rank.
- **Returns**:
  - (int): The base rewards allocated to the given subject.
---

#### **Variables**:
 **`round1PolicyId`**:
   - (string): the unique policy ID for identifying NFTs from Round 1.

 **`round2PolicyId`**:
   - (string): the unique policy ID for identifying NFTs from Round 2.

**`round1Tiers`**:
   - ((int * int) list): A list of thresholds and rewards for Round 1 tiers.

**`round2Tiers`**:
   - ((int * int) list): A list of thresholds and rewards for Round 2 tiers.

In [180]:
let round1PolicyId = "ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d87"
let round2PolicyId = "da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c"

In [181]:
let calculateRewardByTier (rank: int) (tiers: (int * int) list) =
    match tiers |> List.tryFind (fun (threshold, _) -> rank <= threshold) with
    | Some (_, reward) -> reward
    | None -> 0

In [182]:
let round1Tiers = [ (15, 38000); (100, 19250); (250, 13860); (500, 12320); (804, 11550) ]
let round2Tiers = [ (23, 7000); (221, 6000); (664, 5000); (1327, 4500); (2035, 4200) ]

In [183]:
let computeBaseReward (nft: string) (rankData: Map<string, int>) =
    let rank = rankData[nft]
    if nft.Contains(round1PolicyId) then
        calculateRewardByTier rank round1Tiers
    elif nft.Contains(round2PolicyId) then
        calculateRewardByTier rank round2Tiers
    else
        printfn "Unknown NFT policyId for %s" nft
        0

### 3.3 **Identify a Round Two NFT's Minted Order Tier**

This section determines the TEDY bonus allocated to Round 2 Teddy Bear Club NFTs for the [Teddy Bear Surprise](https://medium.com/@TeddySwapDEX/1st-teddy-bear-surprise-945ed9e7405a). For a detailed explanation of the Order Tiers and corresponding TEDY, you may refer to 1.2 - NFT Information above.

---
#### **Function**:
1. **`computeRoundTwoSurprise`**:
- Categorizes an NFT into a tier based on the order of minting.
- **Parameters**:
  - `nft` (string): The subject - Policy ID + Asset Name.
  - `orderedData` (Map<string,int>): A map of subjects and their order of minting.
- **Returns**:
  -(int): The reward corresponding to the NFT’s tier.

In [178]:
let computeRoundTwoSurprise (nft: string) (orderedData: Map<string,int>) = 
    let order = orderedData[nft]
    if order <= 1250 then
        19133
    elif order <= 1400 then
        18133
    elif order < 1600 then
        16800
    elif order = 1600 then
        8400
    elif order < 1800 then
        6400
    elif order = 1800 then
        8400
    elif order <= 1880 then 
        6000
    elif order <= 2000 then 
        8000
    else
        4000

### Computing an NFT's Rank and Reward with Alice’s Teddy Bear

Now that we've built the tools to calculate NFT rankings and determine TEDY rewards, let’s bring these functions to life with a practical example.

Meet Alice, a proud owner of a unique NFT: Teddy Bears Club #5731. Naturally, Alice is curious to find out the rank and TEDY rewards allocated to her her beloved Teddy Bear. Using the tools and functions we’ve developed, we can help Alice uncover this information.

Later on, we will also calculate for the total TEDY tokens Alice can receive. Let’s dive in!

<img src="./assets/teddy5731.png" alt="Teddy Bears Club #5731" width="400" height="400"/>

Let’s start by identifying Alice’s NFT in the system. Alice provides her NFT's subject identifier:


In [192]:
let aliceNft = "ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d8754656464794265617273436c756235373331"

We can then use the `computeBaseRewards` function to determine the NFT's tier and TEDY rewards. Here’s how we process Alice’s NFT:

In [223]:
let aliceBaseReward =  computeBaseReward aliceNft subjectRankData

In [224]:
printfn "Alice's Asset: %s" aliceNft
printfn "Asset's Rank: %d" subjectRankData[aliceNft]
printfn "Asset's TEDY Reward: %d" aliceBaseReward

Alice's Asset: ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d8754656464794265617273436c756235373331
Asset's Rank: 162
Asset's TEDY Reward: 13860


As #5731 is a Round 1 NFT, it is not eligible for the Teddy Bear Surprise. However, Alice's friend Bob does have a Round 2 NFT: Teddy Bear Club #5417

<img src="./assets/teddy5417.png" alt="Teddy Bears Club #5731" width="400" height="400"/>

Let's see determine #5417's NFT Rank and Order, and the corresponding rewards Bob will receive.

In [225]:
let bobNft = "da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c54656464794265617273436c756235343137"

In [226]:
let rankReward =  computeBaseReward ownedNft subjectRankData
let surpriseReward = computeRoundTwoSurprise bobNft subjectOrderData
let bobBaseReward = rankReward + surpriseReward

In [227]:
printfn "Bob's Asset: %s" bobNft
printfn "Asset's Rank: %d" subjectRankData[bobNft]
printfn "Asset's Order: %d" subjectOrderData[bobNft]
printfn "Asset's Rank Reward: %d" rankReward
printfn "Asset's Order Reward: %d" surpriseReward
printfn "Asset's Order Reward: %d" bobBaseReward

Bob's Asset: da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c54656464794265617273436c756235343137
Asset's Rank: 1865
Asset's Order: 239
Asset's Rank Reward: 13860
Asset's Order Reward: 19133
Asset's Order Reward: 32993


### **3.4 Calculate Portion of TEDY Rewards from Surplus TEDY**

In the [initial Round 2 announcements]((https://medium.com/@TeddySwapDEX/teddy-bears-club-round-2-c209556b379f)), surplus rewards were intended to be proportional to the number of NFTs owned. However, the large quantity of remaining tokens diminished the ranking system's impact. To address this, the Swyp Team introduced a tier-based system where TEDY rewards are allocated based on the rarity and ranking of NFTs. The surplus tiers follow the rarity scheme described in section 1.2 - NFT Information. Following the original announcement, Round 1 Teddy Bear Club NFTs receive a bonus of 2.75 in consideration of the higher minting cost.

This section explains the formula used to in this new calculation.

The **Round 1 TEDY Allocation** formula calculates the TEDY allocation for each Round 1 NFT by applying the Round 1 Bonus (2.75x).

$\text{Round 1 TEDY Allocation} = \text{Round 2 TEDY Allocation} * \text{Round 1 Bonus}$

The **Round 2 TEDY Allocation** formula calculates the TEDY allocation for each Round 2 NFT based on the proportion of available token rewards allocated to its tier.

$\text{Round 2 TEDY Allocation} = \frac{\text{Percentage} * \text{Available Token Rewards}}{\text{Round 1 NFTs Sold} * \text{Round 1 Bonus} + \text{Round 2 NFTs Sold}}$


| **Tier** | **Percentage** | **Round 1 TEDY** | **Round 2 TEDY** |
|----------|----------------|------------------|------------------|
| Tier 1   | 10%            | 654,932          | 238,157          |
| Tier 2   | 15%            | 146,196          | 53,162           |
| Tier 3   | 20%            | 98,376           | 35,773           |
| Tier 4   | 27%            | 84,128           | 30,592           |
| Tier 5   | 28%            | 76,310           | 27,749           |


Where:

- **Total INO TEDY Allocation:**  
  The total amount of TEDY designated for Initial NFT Offering (INO) rewards based on the [initial tokenomics](https://medium.com/@TeddySwapDEX/teddyswap-tokenomics-fcd700aea552).  
  $\text{Total INO TEDY Allocation} = 5,000,000,000 \times 4.5\% = 225,000,000 \text{TEDY}$
  
- **Round 1 Bonus:**  
  The bonus multiplier applied to NFTs sold in Round 1.  
 $\text{Round 1 Bonus} = 2.75$

- **Round 1 NFTs Sold:**  
  The total number of NFTs sold in Round 1.  
  $\text{Round 1 NFTs Sold} = 804$

- **Round 2 NFTs Sold:**  
  The total number of NFTs sold in Round 2.  
  $\text{Round 2 NFTs Sold} = 2,035$

- $\text{Tier Tokens}_i$:  
  The number of TEDY tokens allocated per NFT in Tier \(i\).

- $\text{Count of Tier}_i$:
  The number of NFTs in Tier \(i\).

#### **Calculations**

1. **Calculate Distributed Tokens:**

   Determine the total TEDY tokens already distributed based on the tiers of NFTs sold in both rounds.

   $\text{Distributed Tokens} = $
   <br>$\sum_{i=1}^{5} (\text{Round 1 Tier Tokens}_i * \text{Count of Tier}_i) * 2.75 +$ 
   <br>$\sum_{i=1}^{5} (\text{Round 2 Tier Tokens}_i * \text{Count of Tier}_i) + $
   <br>$\sum_{i=1}^{9} (\text{Round 2 MintOrdered Tier Tokens}_i * \text{Count of MintOrdered Tier}_i)$

2. **Determine Available Token Rewards:**

   Calculate the remaining TEDY tokens available for surplus distribution by subtracting the distributed tokens from the total INO TEDY allocation.

   $\text{Available Token Rewards} = \text{Total INO TEDY Allocation} - \text{Distributed Tokens}$

3. **Compute TEDY Surplus per NFT According to Tier:**

   The additional TEDY reward allocated per NFT is calculated by multiplying the allocated percentage for the tier by the available token rewards, then dividing it by the total number of NFTs in that tier. This is multiplied by the Round 1 Bonus for Round 1 NFTs.

   $\text{TEDY Surplus per NFT by Tier} = \frac{\text{Percentage} * \text{Available Token Rewards}}{\text{Round 1 NFTs Sold} * \text{Round 1 Bonus} + \text{Round 2 NFTs Sold}}$

---

#### **Functions**:
1. **`calculateSurplusTedyPerNFT`**:
- Computes the additional TEDY rewards allocated per NFT for each tier, based on the available token rewards, tier allocation percentages, total number of NFTs in each tier, and Round 1 Bonus.
- **Parameters**:
   - `availableTokenRewards` (float) - The total amount of TEDY rewards available for surplus distribution.
- **Returns**:
   - (int list) - A list of TEDY surplus rewards per NFT for each tier, rounded up to the nearest whole number.

---

#### **Variables**:
**`round1Bonus`**:
   - (float): represents the bonus multiplier applied to Round 1 NFTs.

**`round1NFTsSold`**:
   - (int): represents the total NFTs minted for Round 1.

**`round2NFTsSold`**:
   - (int): represents the total NFTs minted for Round 2.
   <br>

**`round1Tier1`, `round1Tier2`, `round1Tier3`, `round1Tier4`, `round1Tier5`**:
   - Tokens allocated per NFT for each tier in Round 1:
     - **round1Tier1**: `30800`
     - **round1Tier2**: `19250`
     - **round1Tier3**: `13860`
     - **round1Tier4**: `12320`
     - **round1Tier5**: `11550`

**`round2Tier1`, `round2Tier2`, `round2Tier3`, `round2Tier4`, `round2Tier5`**:
   - Tokens allocated per NFT for each tier in Round 2:
     - **round2Tier1**: `7000`
     - **round2Tier2**: `6000`
     - **round2Tier3**: `5000`
     - **round2Tier4**: `4500`
     - **round2Tier5**: `4200`

**`round1CountOfTier1`, `round1CountOfTier2`, `round1CountOfTier3`, `round1CountOfTier4`, `round1CountOfTier5`**:
   - The number of NFTs held in each tier:
     - **countOfTier1**: `15`
     - **countOfTier2**: `85`
     - **countOfTier3**: `150`
     - **countOfTier4**: `250`
     - **countOfTier5**: `304`

**`round2CountOfTier1`, `round2CountOfTier2`, `round2CountOfTier3`, `round2CountOfTier4`, `round2CountOfTier5`**:
   - The number of NFTs held in each tier:
     - **countOfTier1**: `23`
     - **countOfTier2**: `198`
     - **countOfTier3**: `443`
     - **countOfTier4**: `663`
     - **countOfTier5**: `708`

**`round2OrderedTier1`, `round2OrderedTier2`, `round2OrderedTier3`, `round2OrderedTier4`, `round2OrderedTier5`, `round2OrderedTier6`, `round2OrderedTier7`, `round2OrderedTier8`, `round2OrderedTier9`**:
   - Tokens allocated per NFT for each tier in Round 2:
     - **round2OrderedTier1**: `19133`
     - **round2OrderedTier2**: `18133`
     - **round2OrderedTier3**: `16800`
     - **round2OrderedTier4**: `8400`
     - **round2OrderedTier5**: `6400`
     - **round2OrderedTier6**: `8400`
     - **round2OrderedTier7**: `6000`
     - **round2OrderedTier8**: `8000`
     - **round2OrderedTier9**: `4000`

**`round2OrderedCountOfTier1`, `round2OrderedCountOfTier2`, `round2OrderedCountOfTier3`, `round2OrderedCountOfTier4`, `round2OrderedCountOfTier5`, `round2OrderedCountOfTier6`, `round2OrderedCountOfTier7`, `round2OrderedCountOfTier8`, `round2OrderedCountOfTier9`**:
   - The number of NFTs held in each tier:
     - **round2OrderedCountOfTier1**: `1250`
     - **round2OrderedCountOfTier2**: `150`
     - **round2OrderedCountOfTier3**: `199`
     - **round2OrderedCountOfTier4**: `1`
     - **round2OrderedCountOfTier5**: `199`
     - **round2OrderedCountOfTier6**: `1`
     - **round2OrderedCountOfTier7**: `80`
     - **round2OrderedCountOfTier8**: `120`
     - **round2OrderedCountOfTier9**: `35`


**`distributedTokens`**:
   - (int): The total tokens distributed across all both rounds.

**`availableTokenRewards`**:
   - (int): The remaining TEDY tokens available for INO rewards after distribution.

**`surplusTierReward`**:
   - (int list): The calculated TEDY surplus reward per NFT for each tier, computed using the `calculateSurplusRewardsPerTier` function.


In [98]:
let round1Bonus = 2.75
let round1NftsSold = 804.0
let round2NftsSold = 2035.0

In [164]:
let round1Tier1 = 30800.0
let round1Tier2 = 19250.0
let round1Tier3 = 13860.0
let round1Tier4 = 12320.0
let round1Tier5 = 11550.0

let round1CountOfTier1 = 15.0
let round1CountOfTier2 = 85.0
let round1CountOfTier3 = 150.0
let round1CountOfTier4 = 250.0
let round1CountOfTier5 = 304.0

let round2Tier1 = 7000.0
let round2Tier2 = 6000.0
let round2Tier3 = 5000.0
let round2Tier4 = 4500.0
let round2Tier5 = 4200.0

let round2CountOfTier1 = 23.0
let round2CountOfTier2 = 198.0
let round2CountOfTier3 = 443.0
let round2CountOfTier4 = 663.0
let round2CountOfTier5 = 708.0

let round2OrderedTier1 = 19133.0
let round2OrderedTier2 = 18133.0
let round2OrderedTier3 = 16800.0
let round2OrderedTier4 = 8400.0
let round2OrderedTier5 = 6400.0
let round2OrderedTier6 = 8400.0
let round2OrderedTier7 = 6000.0
let round2OrderedTier8 = 8000.0
let round2OrderedTier9 = 4000.0

let round2OrderedCountOfTier1 = 1250.0
let round2OrderedCountOfTier2 = 150.0
let round2OrderedCountOfTier3 = 199.0
let round2OrderedCountOfTier4 = 1.0
let round2OrderedCountOfTier5 = 199.0
let round2OrderedCountOfTier6 = 1.0
let round2OrderedCountOfTier7 = 80.0
let round2OrderedCountOfTier8 = 120.0
let round2OrderedCountOfTier9 = 35.0

let totalTokenRewards = 225000000

In [165]:
let round1Counts = [| round1CountOfTier1; round1CountOfTier2; round1CountOfTier3; round1CountOfTier4; round1CountOfTier5 |]
let round1Tiers = [| round1Tier1; round1Tier2; round1Tier3; round1Tier4; round1Tier5 |]
let round2Counts = [| round2CountOfTier1; round2CountOfTier2; round2CountOfTier3; round2CountOfTier4; round2CountOfTier5 |]
let round2Tiers = [| round2Tier1; round2Tier2; round2Tier3; round2Tier4; round2Tier5 |]
let round2OrderedTiers = [| 
    round2OrderedTier1; 
    round2OrderedTier2; 
    round2OrderedTier3; 
    round2OrderedTier4; 
    round2OrderedTier5; 
    round2OrderedTier6; 
    round2OrderedTier7; 
    round2OrderedTier8; 
    round2OrderedTier9 
|]

let round2OrderedCounts = [| 
    round2OrderedCountOfTier1; 
    round2OrderedCountOfTier2; 
    round2OrderedCountOfTier3; 
    round2OrderedCountOfTier4; 
    round2OrderedCountOfTier5; 
    round2OrderedCountOfTier6; 
    round2OrderedCountOfTier7; 
    round2OrderedCountOfTier8; 
    round2OrderedCountOfTier9 
|]


In [166]:
let distributedTokens = 
    Math.Ceiling(
        (Array.zip round1Counts round1Tiers |> Array.sumBy (fun (count, tier) ->  count *  tier)) * round1Bonus +
        (Array.zip round2Counts round2Tiers |> Array.sumBy (fun (count, tier) ->  count *  tier)) +
        (Array.zip round2OrderedCounts round2OrderedTiers |> Array.sumBy (fun (count, tier) ->  count *  tier))
    )

In [167]:
let availableTokenRewards =  totalTokenRewards - int distributedTokens

In [207]:
let calculateSurplusRewardsPerTier (availableTokenRewards: float) =
    let percentages = [0.1; 0.15; 0.2; 0.27; 0.28]
    let nftCounts = [round1CountOfTier1 * round1Bonus + round2CountOfTier1;
                     round1CountOfTier2 * round1Bonus + round2CountOfTier2;
                     round1CountOfTier3 * round1Bonus + round2CountOfTier3;
                     round1CountOfTier4 * round1Bonus + round2CountOfTier4;
                     round1CountOfTier5 * round1Bonus + round2CountOfTier5]
    percentages
    |> List.mapi (fun i percentage ->
        let rewardPerNFT = (availableTokenRewards * percentage) / float nftCounts.[i]
        Math.Ceiling(rewardPerNFT) |> int
    )

In [209]:
let surplusTierReward = calculateSurplusRewardsPerTier availableTokenRewards
let roundOneSurplusRewardsPerTier = 
    surplusTierReward 
    |> List.map (fun rewards -> int (Math.Ceiling(float rewards * 2.75)))


## **3.5 Calculate Total TEDY Rewards** 
In this step, we query the [CardanoScan API](https://docs.cardanoscan.io/) for the NFTs for a certain address, and calculate the total TEDY rewards for a given address based on the Teddy Bear Club NFTs it holds.

This process involves **generating the JSON** file - `collectionassets.json` - containing the address's token data by following the workflow outlined below.

`collectionassets.json`, is created by querying the [CardanoScan API](https://docs.cardanoscan.io/). The JSON file is not included in this repository, and you must generate it yourself if you wish to explore these calculations. 

Once the token data is available, we use the `computeBaseRewards` and `computeSurplusRewards` function to compute the total TEDY rewards for the address.

*Note!*
You will need to replace the placeholder with the address you wish to query.

---

### 3.5.1 CardanoScan API Query

*Note!*<br>
To use the [CardanoScan API](https://docs.cardanoscan.io/), you must have a valid API key. The `getApiKey` function retrieves this key from your environment variables, so ensure your key is named `CARDANO_API_KEY` or update the function to match your setup. 

1. **`getApiKey`**:
   - Retrieves your API key from the environment.
   - Ensures the key is securely accessed and avoids hardcoding sensitive information.

2. **`saveJsonToFile`**:
   - Formats the JSON response into a more readable format and saves it to a file.
   - By default, it saves the output to [`collectionassets.json`](./data/collectionassets.json).

3. **`getAssetsByAddress`**:
   - Configures the API request by setting the required headers (including the API key) and query parameters.
   - Sends the request to the API endpoint and handles the response.

---

In [210]:
type Token = {
    policyId: string
    assetName: string
    fingerprint: string
    assetId: string
    balance: string
}

type ApiResponse = {
    tokens: Token list
    count: int
    pageNo: int
    limit: int
}

In [171]:
let getApiKey () =
    match Environment.GetEnvironmentVariable("CARDANO_API_KEY") with
    | null -> failwith "API key not found. Set it in your environment variables."
    | key -> key

In [172]:
let saveJsonToFile (filePath: string) (jsonResponse: string) =
    try
        
        let data = JsonSerializer.Deserialize<obj>(jsonResponse)
        let options = JsonSerializerOptions(WriteIndented = true, Encoder = System.Text.Encodings.Web.JavaScriptEncoder.UnsafeRelaxedJsonEscaping)
        let jsonOutput = JsonSerializer.Serialize(data, options)

        File.WriteAllText(filePath, jsonOutput)
    with ex ->
        printfn "Failed to save JSON at %s. Error: %s" filePath ex.Message

In [173]:
let getAssetsByAddress (address: string) (filePath: string) =
    async {
        let apiKey = getApiKey()
        let baseUrl = "https://api.cardanoscan.io/api/v1/asset/list/byAddress"
        let mutable pageNo = 1
        let mutable allTokens = []
        let mutable hasMoreData = true

        use client = new HttpClient()
        client.DefaultRequestHeaders.Add("apiKey", apiKey)

        try
            while hasMoreData do
                let url = $"{baseUrl}?address={address}&pageNo={pageNo}"
                let! response = client.GetAsync(url) |> Async.AwaitTask

                if response.IsSuccessStatusCode then
                    let! content = response.Content.ReadAsStringAsync() |> Async.AwaitTask
                    try
                        let data = JsonSerializer.Deserialize<ApiResponse>(content)

                        if data.tokens.IsEmpty then
                            hasMoreData <- false
                        else
                            allTokens <- allTokens @ data.tokens
                            pageNo <- pageNo + 1
                    with ex ->
                        printfn "JSON Deserialization Error: %s" ex.Message
                        printfn "Raw Content: %s" content
                        hasMoreData <- false
                        ()
                else
                    let! errorContent = response.Content.ReadAsStringAsync() |> Async.AwaitTask
                    printfn "API Error: %s" errorContent
                    hasMoreData <- false
                    return ()

            if allTokens.IsEmpty then
                printfn "No tokens found for address: %s" address
                return ()

            let assetIds =
                allTokens
                |> List.filter (fun token -> token.assetId.Contains(round1PolicyId) || token.assetId.Contains(round2PolicyId))
                |> List.map (fun token -> token.assetId)

            let allTokensJson = JsonSerializer.Serialize(allTokens, JsonSerializerOptions(WriteIndented = true))
            saveJsonToFile filePath allTokensJson

            printfn "Successfully saved all tokens to %s" filePath
            return Ok (), assetIds
        with ex ->
            printfn "Exception occurred: %s" ex.Message
            return Error $"Exception: {ex.Message}", []
    }

### 3.5.2 Reward Calculations

#### **Functions**:
1. **`computeSurplusRewards`**:
   - Computes the additional TEDY rewards allocated to a given NFT based on its round and tier.
   - **Parameters**:
     - **`nft`** (string): The unique identifier of the NFT.
     - **`rankData`** (Map<string, int>): A mapping of NFT IDs to their ranks.
     - **`surplusRewardsPerTier`** (list<int>): A list of surplus rewards allocated per NFT in each tier.
   - **Returns**:
     - (int): The calculated surplus reward for the NFT.

2. **`calculateTotalRewards`**:
  - Computes the total TEDY rewards for all NFTs owned by an address.
  - **Parameters**:
    - **`nftList`** (list<string>): A list of NFTs held by the address.
    - **`round1PolicyId`** (string): The policy ID for tokens in Round 1.
    - **`round2PolicyId`** (string): The policy ID for tokens in Round 2.
    - **`subjectRankData`** (Map<string, int>): A mapping of token IDs to their ranks.
    - **`surplusRewardsPerTier`** (list<int>): A list of surplus rewards per NFT for each tier.
  - **Returns**:
    - (int): The total calculated TEDY rewards for all NFTs owned by the address.

---

#### **Variable**:
- **`totalTedyRewards`**: 
  - (int) — Total TEDY rewards allocated to the address. This calculation assumes the original 5,000,000,000 market cap.

In [174]:
let computeSurplusRewards (nft: string) (rankData: Map<string, int>) (surplusRewardsPerTier: int list) =
    let rank = rankData[nft]
    let roundOneSurplusRewardsPerTier = 
        surplusRewardsPerTier 
        |> List.map (fun rewards -> int (Math.Ceiling(float rewards * 2.75)))
    if nft.Contains(round1PolicyId) then
        calculateRewardByTier rank (List.zip [15; 100; 250; 500; 804] roundOneSurplusRewardsPerTier)
    elif nft.Contains(round2PolicyId) then
        calculateRewardByTier rank (List.zip [23; 221; 664; 1327; 2035] surplusRewardsPerTier)
    else
        -1

In [175]:
let calculateTotalRewards (nftList: list<string>) (round1PolicyId: string) (round2PolicyId: string) (subjectRankData: Map<string, int>) (surplusRewardsPerTier: int list) =
    let rewardsList =
        [ for nft in nftList do
            match nft.Contains(round1PolicyId) || nft.Contains(round2PolicyId) with
            | true ->
                let baseReward =  
                    if nft.Contains(round1PolicyId) then
                        computeBaseReward nft subjectRankData
                    elif nft.Contains(round2PolicyId) then
                        computeBaseReward nft subjectRankData + computeRoundTwoSurprise nft subjectOrderData
                    else
                        0
                let surplusReward = computeSurplusRewards nft subjectRankData surplusRewardsPerTier
                yield baseReward + surplusReward
            | false -> ()
        ]

    let totalTedyRewards = rewardsList |> List.sum
    totalTedyRewards

### Enter Your Cardano Address

Replace the placeholder address in the code below with the **hex representation of the Cardano address** you want to query.

In [176]:
let address = "01a2372d4f613b6d64da77f6047d4756b261b31619a54ec32e49b09797cfa1b556bfa358a865b6982a6e3a8c5f2e7144a430b1c7d9173c2ea4"

In [239]:
let fetchRewardsAsync () =
    async {
        let filePath = "./data/collectionassets.json"

        let! result, assetIds = getAssetsByAddress address filePath

        match result with
        | Ok () ->
            let totalTedyRewards = calculateTotalRewards assetIds round1PolicyId round2PolicyId subjectRankData surplusTierReward
            return Some(assetIds, totalTedyRewards)
        | Error errMsg ->
            printfn "Error: %s" errMsg
            return None
    }


In [242]:
let mutable totalTedyRewards = 0
let result = fetchRewardsAsync () |> Async.RunSynchronously

match result with
| Some (assetIds, rewards) ->
    totalTedyRewards <- rewards
    printfn "Assets: %A" assetIds
    printfn "Total Rewards: %d" totalTedyRewards
| None ->
    printfn "Failed to fetch rewards."


Successfully saved all tokens to ./data/collectionassets.json
Assets: ["da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c54656464794265617273436c756235343137"]
Total Rewards: 51082


### **4. Adjust Rewards for Market Cap**
In this step, we adjust the final TEDY reward allocation to reflect the updated market cap of 8,000,000, ensuring alignment with the revised tokenomics. The adjustment scales the rewards originally calculated for the initial market cap of 5,000,000,000.

Where:
- Total TEDY Allocation: The total rewards calculated based upon the initial marketcap.
- [Current Marketcap](https://medium.com/@TeddySwapDEX/tedy-tokenomics-v2-fc92723ead0c): 8,000,000
- [Previous Market Cap](https://medium.com/@TeddySwapDEX/teddyswap-tokenomics-fcd700aea552): 5,000,000

$\text{Final Round Reward} = (\frac{\text{Total TEDY Allocation} * \text{Current Marketcap}}{\text{Previous Market Cap}})$

---

#### **Functions**:
1. **`marketcapAdjustment`**:
- Adjusts the reward amount based on the ratio of the current market cap to the past market cap.
- **Parameters**:
   - `rewards` (float): The original reward amount to be adjusted.
- **Returns**:
   - (int): The adjusted reward amount.

---

#### **Variables**:
**`pastMarketCap`**:
   - The previous market cap of the token, used as a reference for adjustments.

**`currentMarketCap`**:
   - The current market cap of the token, used to scale rewards.

**`totalTedyReward`**:
   - The final TEDY rewards allocation 


In [232]:
let initialMarketCap = 5000000000.0
let currentMarketCap = 8000000.0

In [233]:
let marketcapAdjustment (rewards: float) = 
    let finalRoundReward = (rewards*currentMarketCap)/initialMarketCap
    Math.Ceiling(finalRoundReward)

In [243]:
let totalTedyReward = int(marketcapAdjustment totalTedyRewards)

# Alice's Total TEDY Rewards

Now that we’ve developed the functions to calculate rewards, it’s time to discover the total TEDY rewards Alice will receive. Unfortunately, we don't have Alice's address, so we will be running through the calculations manually - to provide an example to follow.

We saw above that as a Round 1 NFT, ranked 162 and placed in Tier 3, #5731 earns Alice a total of **13,860 TEDY tokens**. Furthermore, she gains a surplus reward of **98,376**. 

Her total reward is the sum of the two, 13,860 + 98,376  = **112,236**.

Finally, we scale her rewards according to the new market cap of 8,000,000.

$\text{Final Round Reward} = (\frac{\text{112,236} * \text{8,000,000}}{\text{5,000,000,000}}) = 180$

In [249]:
let surplusReward = computeSurplusRewards ownedNFT subjectRankData surplusTierReward
let totalReward = aliceBaseReward + surplusReward
let finalReward = int(marketcapAdjustment totalReward)

In [250]:
printfn "#5731 Base Reward: %d" aliceBaseReward
printfn "Alice's Initial TEDY Rewards: %d" totalReward
printfn "Alice's Total TEDY Rewards: %d" (int finalReward)

#5731 Base Reward: 13860
Alice's Initial TEDY Rewards: 112236
Alice's Total TEDY Rewards: 180


# Calculate Your Rewards

This section allows you to calculate the TEDY reward allocation by providing either a Cardano address or a list of subjects.

---
### Option 1: Provide a Cardano Address
Replace the placeholder address in the code below with the **hexadecimal representation** of the Cardano address you want to query. This will fetch all associated NFTs and calculate the rewards accordingly.

In [261]:
let address = "addr1qx3rwt20vyak6ex6wlmqgl2826exrvckrxj5asewfxcf09705x64d0artz5xtd5c9fhr4rzl9ec5ffpsk8raj9eu96jqqw2njm"

In [ ]:
let result = fetchRewardsAsync () |> Async.RunSynchronously

match result with
| Some (assetIds, rewards) ->
    totalTedyRewards <- rewards
    printfn "Assets: %A" assetIds
    printfn "Total TEDY Pre-MarketCap Adjustment: %d" totalTedyRewards
    printfn "Total TEDY Post-MarketCap Adjustment: %d" (int (marketcapAdjustment totalTedyRewards))
| None ->
    printfn "Failed to fetch rewards."

### Option 2: Provide a List of Subjects

If you already have the specific NFT subjects, you can directly input them into the list. Each subject should be enclosed in quotes and separated by a **semicolon** (;).

i.e:<br>
[<br> "ab182ed76b669b49ee54a37dee0d0064ad4208a859cc4fdf3f906d8754656464794265617273436c756235373331";<br>
"da3562fad43b7759f679970fb4e0ec07ab5bebe5c703043acda07a3c54656464794265617273436c756235343137"<br>
]

In [258]:
let subjectList = 
    [
        
    ]

In [ ]:
let preadjustmentTedy = calculateTotalRewards subjectList round1PolicyId round2PolicyId subjectRankData surplusTierReward
let postadjustmentTedy = int (marketcapAdjustment preadjustmentTedy)

printfn "Total TEDY Pre-MarketCap Adjustment: %d" preadjustmentTedy
printfn "Total TEDY Post-MarketCap Adjustment: %d" postadjustmentTedy

## **Disclaimers**
1. *Market Cap Adjustments*:
The calculations above are based on the initial market cap of **5,000,000,000** used to model events during the INO. `marketcapAdjustment` scales rewards to align with the updated market cap of **8,000,000**, resulting in adjusted and lower TEDY reward amounts.

The figures provided are *estimates* and are intended for informational purposes only. We do not guarantee their accuracy and disclaim liability for any investment decisions made based on this information. Use this data at your own risk.

2. Exclusions:
These NFT calculations **do not** account for the following bonuses promised by TeddySwap:

- FISO (Fair Initial Stake Offering) bonuses
- ITN (Incentivized Testnet) rewards
- LBE (Liquidity Bootstrapping Event) rewards
- Yield farming bonuses
